# Random Forest Classifier

In [25]:
#Libraries
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn.externals.six import StringIO
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV

In [29]:
def create_kaggle_response(classifier, csv_name):
    grid_ans_cv = classifier.predict_proba(test_df)
    p1 = [ans[0] for ans in grid_ans_cv]
    p2 = [ans[1] for ans in grid_ans_cv]
    p3 = [ans[2] for ans in grid_ans_cv]
    answer_svm_cv_folds = pd.DataFrame({
        "listing_id": listing_ids,
        "high": p1,
        "medium": p2,
        "low": p3
    })
    answer_svm_cv_folds.to_csv(csv_name, index=False)

In [30]:
#Features
features = [
    'bathrooms', 
    'bedrooms', 
    'latitude', 
    'longitude', 
    'price',
    'Special Characters',
    'Numbers', 
    'StopWord',
    'Uppercase',
]

In [31]:
#Reading file and setting X & y. The features and labels. Here we are dropping the unwanted features for our classifiers
df = pd.read_json('cleaned_train.json')

df_X = df.drop('interest_level', axis=1)
df_y = df['interest_level']

df_X = df.drop('interest_level', axis=1)
df_X = df_X.drop('description', axis=1)
df_X = df_X.drop('display_address', axis=1)
df_X = df_X.drop('street_address', axis=1)
df_X = df_X.drop('features', axis=1)
df_X = df_X.drop('created', axis=1)
df_X = df_X.drop('listing_id', axis=1)


In [32]:
#Importing KFold
from sklearn.model_selection import KFold # import KFold

kf = KFold(n_splits=5)
kf.get_n_splits(df_X)
KFold(n_splits=5, random_state=None, shuffle=False)

KFold(n_splits=5, random_state=None, shuffle=False)

In [33]:
df.head()

,bathrooms,bedrooms,created,description,display_address,features,latitude,listing_id,longitude,price,street_address,interest_level,Special Characters,Numbers,StopWord,Uppercase
4,1.0,1,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,2400,145 Borinquen Place,medium,20,2,8,45
6,1.0,2,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,3800,230 East 44th,low,9,2,40,71
9,1.0,2,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,3495,405 East 56th Street,medium,11,2,28,62
10,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,3000,792 Metropolitan Avenue,medium,8,1,8,61
15,1.0,0,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,2795,340 East 34th Street,low,6,1,0,36


In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)
#Maybe remove test_size and set random states to 1

In [35]:
from sklearn.ensemble import RandomForestClassifier

RFM = RandomForestClassifier(n_estimators=200, random_state=0, criterion='gini', max_depth=80, min_samples_split=2, min_samples_leaf=4, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
RFM.fit(X_train, y_train)
y_pred = RFM.predict(X_test)

In [43]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

scores = cross_val_score(estimator = RFM, X = df_X, y = df_y, cv = 10, n_jobs = 4)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred) )
print(classification_report(y_test, y_pred) )
print("Overall f-score:", f1_score(y_test, y_pred, average='micro') )
print("Score", scores)

Accuracy: 0.7151595473544704
              precision    recall  f1-score   support

        high       0.59      0.21      0.31       811
         low       0.75      0.94      0.84      6696
      medium       0.47      0.24      0.31      2302

    accuracy                           0.72      9809
   macro avg       0.60      0.46      0.49      9809
weighted avg       0.67      0.72      0.67      9809

Overall f-score: 0.7151595473544704
Score [0.72171254 0.73633768 0.72206362 0.73205546 0.72267537 0.72185971
 0.7230832  0.71411093 0.71676183 0.71941272]


In [41]:
test_df = pd.read_json('test2.json')
test_df_unformatted = pd.read_json('test.json')
listing_ids = test_df_unformatted['listing_id']

In [42]:
create_kaggle_response(classifier = RFM, csv_name = 'rfm.csv')

In [11]:
df_y2 = pd.get_dummies(df_y)
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y2, test_size = 0.2, random_state = 0)

In [44]:
test_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 200, num = 3)],
               'max_features': ['auto', 'sqrt'],
               'min_samples_split': [1, 3, 5],
               'min_samples_leaf': [1, 3, 5],
               'bootstrap': [True, False]}

In [47]:
from sklearn.model_selection import GridSearchCV

RFR = RandomForestClassifier()
RFR1 = RandomizedSearchCV(estimator = RFR, param_distributions = test_grid, n_iter = 100, cv = 3, verbose = 1, random_state = 30, n_jobs = -1)
RFR1.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
RFR1.best_params_

# Normalized Random Forest

In [157]:
######### normalizing

from sklearn import preprocessing

x = df_X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_X_normalized = pd.DataFrame(x_scaled)


X_train2, X_test2, y_train2, y_test2 = train_test_split(df_X_normalized, df_y, random_state=1)


sc_normal = StandardScaler()
X_train2 = sc_normal.fit_transform(X_train2)
X_test2 = sc_normal.transform(X_test2)


RFM_normal = RandomForestClassifier(n_estimators=20, random_state=0)
RFM_normal.fit(X_train2, y_train2)
y_pred2 = RFM_normal.predict(X_test2)

#Normalized score
print("Accuracy:",metrics.accuracy_score(y_test2, y_pred2) )
print(classification_report(y_test2, y_pred2) )
print("Overall f-score:", f1_score(y_test, y_pred, average='weighted') )

Accuracy: 0.7160916727836228
              precision    recall  f1-score   support

        high       0.46      0.27      0.34       958
         low       0.78      0.90      0.84      8568
      medium       0.44      0.29      0.35      2735

    accuracy                           0.72     12261
   macro avg       0.56      0.49      0.51     12261
weighted avg       0.68      0.72      0.69     12261

f-score: 0.6811717106628986


In [134]:
test_df = pd.read_json('test2.json')
test_df_unformatted = pd.read_json('test.json')
listing_ids = test_df_unformatted['listing_id']

create_kaggle_response(classifier=RFM, csv_name='rfm_norm.csv')